# Underworld3 Scaling System - Final Implementation

This notebook demonstrates the completed automatic scaling system that allows users to specify problems in convenient geological units while solvers work with O(1) values for optimal numerical conditioning.

## Key Features
- **Context Manager**: `uw.scaled_symbols()` controls when scaling is applied
- **Unwrap Integration**: Scaling applied during `unwrap()` process
- **Hierarchical Support**: Works with UW expressions containing multiple variables
- **Automatic Discovery**: Finds all variables with units in expressions

In [ ]:
import sys
sys.path.insert(0, '/Users/lmoresi/+Underworld/underworld-pixi-2/underworld3/src')

import underworld3 as uw
from underworld3.discretisation.enhanced_variables import EnhancedMeshVariable
from underworld3.function.expressions import unwrap

print("🌍 Underworld3 Scaling System Demo")
print("=" * 35)

## Setup: Variables with Geological Units

In [ ]:
# Create mesh
mesh = uw.meshing.StructuredQuadBox(elementRes=(4, 4), minCoords=(0, 0), maxCoords=(1000, 1000))

# Temperature in Kelvin (typical mantle temperatures)
temperature = EnhancedMeshVariable(
    "temp", mesh, 1, 
    units="K", 
    units_backend="sympy", 
    varsymbol=r"T"
)
temperature.set_reference_scaling(1500.0)  # Typical mantle temperature

# Velocity in cm/year (geological rates)
velocity = EnhancedMeshVariable(
    "vel", mesh, 2, 
    units="cm/year", 
    units_backend="sympy", 
    varsymbol=r"\mathbf{v}"
)
velocity.set_reference_scaling(5.0)  # Typical plate motion

print(f"Temperature scale factor: {temperature.scale_factor}")
print(f"Velocity scale factor: {velocity.scale_factor}")
print(f"Velocity units: {velocity.units}")

## Test 1: Simple Variable Scaling

In [ ]:
print("🔬 Test 1: Simple Variable Scaling")
print("" + "-" * 35)

# Create simple expression
simple_expr = temperature.sym
print(f"Expression: {simple_expr}")

# Normal unwrap (no scaling)
normal_result = unwrap(simple_expr)
print(f"Normal unwrap: {normal_result}")

# Scaled unwrap
with uw.scaled_symbols():
    scaled_result = unwrap(simple_expr)
    print(f"Scaled unwrap: {scaled_result}")

print(f"Scaling applied: {normal_result != scaled_result}")

## Test 2: Mathematical Expressions

In [ ]:
print("\n🔬 Test 2: Mathematical Expressions")
print("-" * 36)

# Create mathematical expression
math_expr = 2 * temperature + 0.5 * velocity[0]
print(f"Expression: {math_expr}")

# Normal unwrap
normal_math = unwrap(math_expr)
print(f"Normal unwrap: {normal_math}")

# Scaled unwrap
with uw.scaled_symbols():
    scaled_math = unwrap(math_expr)
    print(f"Scaled unwrap: {scaled_math}")

print(f"Scaling applied: {normal_math != scaled_math}")

## Test 3: UW Expressions (Hierarchical)

In [ ]:
print("\n🔬 Test 3: UW Expression (Hierarchical)")
print("-" * 40)

# Create UW expression with multiple variables
uw_expr = uw.function.expression(
    "heat_equation", 
    temperature.sym + velocity[0].sym + velocity[1].sym, 
    "Complex heat equation with advection"
)
print(f"UW Expression: {uw_expr}")

# Normal unwrap
normal_uw = unwrap(uw_expr)
print(f"Normal unwrap: {normal_uw}")

# Scaled unwrap
with uw.scaled_symbols():
    scaled_uw = unwrap(uw_expr)
    print(f"Scaled unwrap: {scaled_uw}")

print(f"Scaling applied: {normal_uw != scaled_uw}")

## Test 4: Complex Expression with Mixed Units

In [ ]:
print("\n🔬 Test 4: Complex Mixed Units Expression")
print("-" * 42)

# Add pressure variable
pressure = EnhancedMeshVariable(
    "press", mesh, 1, 
    units="GPa", 
    units_backend="sympy", 
    varsymbol=r"P"
)
pressure.set_reference_scaling(1.0)  # Typical mantle pressure range

# Create complex expression
complex_expr = uw.function.expression(
    "coupled_system",
    (temperature.sym / 1000) + (velocity[0].sym * 1e6) + pressure.sym,
    "Coupled thermal-mechanical system"
)

print(f"Complex Expression: {complex_expr}")
print(f"Pressure scale factor: {pressure.scale_factor}")

# Normal unwrap
normal_complex = unwrap(complex_expr)
print(f"Normal unwrap: {normal_complex}")

# Scaled unwrap
with uw.scaled_symbols():
    scaled_complex = unwrap(complex_expr)
    print(f"Scaled unwrap: {scaled_complex}")

print(f"All scale factors automatically applied: {normal_complex != scaled_complex}")

## Summary

✅ **Scaling System Complete**

The redesigned scaling system successfully:

1. **Applies scaling during unwrap**: Variables maintain normal symbols, scaling applied when context is active
2. **Works with UW expressions**: Hierarchical expressions capture scaling at unwrap time
3. **Handles multiple variables**: Automatically finds all variables with units in complex expressions
4. **Maintains backward compatibility**: No impact on existing code, purely additive
5. **JIT compilation ready**: Scaled expressions produce standard SymPy that compiles normally

### Usage Pattern
```python
# Create variables with units
temperature = EnhancedMeshVariable("T", mesh, 1, units="K")
temperature.set_reference_scaling(1500.0)

# Build expressions normally
expr = uw.function.expression("heat_eq", 2 * temperature, "description")

# Apply scaling during compilation
with uw.scaled_symbols():
    scaled_expr = unwrap(expr)  # Contains scale factors
```

This design allows users to work in convenient geological units while ensuring solvers receive properly conditioned O(1) expressions.